# Prep

## Imports

In [169]:
import pandas as pd
import numpy as np
import requests
import pprint
import json
import os
import re

#from dotenv import load_dotenv
from bs4 import BeautifulSoup

## Auth

In [170]:
#load_dotenv(r"C:\\Users\\User\\Documents\\GitHub\\movies\\tmdb_auth.env")

#api_key = os.getenv("API_KEY")
#access_token = os.getenv("ACCESS_TOKEN")

# TMDB API Data

In [171]:
hold = '''
tmdb_url = "https://api.themoviedb.org/3/account/21623434/rated/movies?language=en-US&page=1&sort_by=created_at.asc"

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {access_token}"
}
'''

#response = requests.get(tmdb_url, headers=headers)


In [172]:
#data = json.loads(response.text)
#data['results']

# Scrape Letterboxd Data

## Functions

### Extraction and general structuring

In [173]:
# reads through a Letterboxd list and gets the url for each movie in it

def get_film_urls(list_url):
    content = requests.get(list_url).text
    soup = BeautifulSoup(content, 'html')

    url_list = [div['data-target-link'] for div in soup.find_all('div', class_='film-poster')]

    return url_list

In [174]:
# extracts the complete, raw HTML fom a URL

def get_raw_film_html(film_url):
    url = "https://letterboxd.com" + film_url
    content = requests.get(url).text
    soup = BeautifulSoup(content, 'html.parser')

    return soup

In [175]:
# goes through the raw HTML. extracts and structures general data and metadata about the film 

def get_general_film_data(soup):
    duration_string = soup.find(class_='text-footer').get_text().replace('\xa0', ' ').strip()

    general_data = {
        'letterboxd_id': soup.find(id='backdrop')['data-film-id'],
        'letterboxd_shorttitle': soup.find('h1', class_='filmtitle').get_text(),
        'letterboxd_longtitle': soup.find(property='og:title')['content'],
        'letterboxd_slug': soup.find(id='backdrop')['data-film-slug'],
        'letterboxd_url': soup.find(property='og:url')['content'],
        'imdb_url': soup.find('a', {'data-track-action': 'IMDb'})['href'],
        'tmdb_url': soup.find('a', {'data-track-action': 'TMDb'})['href'],
        'tmdb_id': '',
        'release_year': soup.find(class_='releaseyear').find('a').get_text(strip=True),
        'duration': re.search(r'(\d+)\s+mins', duration_string).group(1),
        'avg_rating': soup.find('meta', attrs={'name': 'twitter:data2'})['content'].split(' out')[0]
    }

    general_data['tmdb_id'] = general_data['tmdb_url'].split('/')[-2]

    return general_data

In [176]:
# goes through the raw HTML. extracts and structures data about the movie's cast

def get_film_cast(soup):
    cast_list = []
    cast = soup.find(name='div', class_='cast-list').find_all('a', class_='tooltip')

    for member in cast:
        cast_member_info = {
            'name': member.get_text(strip=True),
            'link': member['href']
            #'character_name': member['title']
        }

        try:
            cast_member_info['character_name'] = member['title']
        except:
            cast_member_info['character_name'] = None
        cast_list.append(cast_member_info)

    return cast_list

In [177]:
# goes through the raw HTML. extracts and structures data about the movie's crew

def get_film_crew(soup):
    crew_list = []
    crew = soup.find(id='tab-crew').find_all('a')

    for member in crew:
        split_link = member['href'].split('/')
        
        crew_member_info = {
            'name': member.get_text(strip=True),
            'role': split_link[1],
            'link': member['href'],
        }
        crew_list.append(crew_member_info)
    
    return crew_list

In [178]:
# goes through the raw HTML. extracts and structures data about other details concerning the movie

def get_film_details(soup):
    details_list = []
    details = soup.find(id='tab-details').find_all('a')

    for detail in details:
        split_link = detail['href'].split('/')

        detail_info = {
            'key': '',
            'value': detail.get_text(strip=True),
            'link': detail['href']
        }

        if 'studio' in detail['href']:
            detail_info['key'] = 'studio'
        elif 'country' in detail['href']:
            detail_info['key'] = 'country'
        elif 'language' in detail['href']:
            detail_info['key'] = 'language'
        else:
            detail_info['key'] = 'ERROR'
        details_list.append(detail_info)

    return details_list

In [179]:
# goes through the raw HTML. extracts and structures data about the movie's genres and themes

def get_film_genres(soup):
    genres = [a_tag.get_text(strip=True) for a_tag in soup.find(id='tab-genres').find_all('a')]

    return genres[:-1]

In [180]:
# creates a loop using the previous functions to extract all the relevant data and unify it in a dict

def get_complete_film_data(film_url):
    film_soup = get_raw_film_html(film_url)

    film_data = {
        'general_data': get_general_film_data(film_soup),
        'cast': get_film_cast(film_soup),
        'crew': get_film_crew(film_soup),
        'details': get_film_details(film_soup),
        'genres_and_themes': get_film_genres(film_soup)
    }

    return film_data

In [181]:
# loops through all URLs in a list, extracting and structuring data from all of them

def get_all_films(url_list):
    whole_data = []

    counter = 0
    for film in url_list:
        #print(f"Extracting from URL #{counter}:\n{film}\n")
        whole_data.append(get_complete_film_data(film))
        counter += 1
    
    return whole_data

In [182]:
# transforms the data dictionaries into dataframes

def dicts_to_dfs(data):
    all_dfs_gdata = [] # general data
    all_dfs_cast = []
    all_dfs_crew = []
    all_dfs_details = []
    all_dfs_gthemes = []


    for film in data:
        id = film['general_data']['letterboxd_id']
        title = film['general_data']['letterboxd_shorttitle']
        
        single_df_gdata = pd.DataFrame.from_dict([film['general_data']])
        all_dfs_gdata.append(single_df_gdata)

        single_df_cast = pd.DataFrame.from_dict(film['cast']).assign(film_id = id, film_title = title)
        all_dfs_cast.append(single_df_cast)

        single_df_crew = pd.DataFrame.from_dict(film['crew']).assign(film_id = id, film_title = title)
        all_dfs_crew.append(single_df_crew)

        single_df_details = pd.DataFrame.from_dict(film['details']).assign(film_id = id, film_title = title)
        all_dfs_details.append(single_df_details)

        single_df_gthemes = pd.DataFrame.from_dict(film['genres_and_themes']).assign(film_id = id, film_title = title)
        all_dfs_gthemes.append(single_df_gthemes)

    all_dfs_dict = {
        'df_gdata': pd.concat(all_dfs_gdata),
        'df_cast': pd.concat(all_dfs_cast),
        'df_crew': pd.concat(all_dfs_crew),
        'df_details': pd.concat(all_dfs_details),
        'df_gthemes': pd.concat(all_dfs_gthemes)
    }

    return all_dfs_dict

### Other treatments

# Extract, create and treat DFs

In [183]:
#film_urls = get_film_urls("https://letterboxd.com/dromemario/list/fff-film-fueled-friends/")

#films_data = get_all_films(film_urls)

#with open("films_data.json", "w") as json_file:
#    json.dump(films_data, json_file, indent=4)

In [184]:
with open("films_data.json", "r") as json_file:
    films_data = json.load(json_file)

In [185]:
all_dfs_dict = dicts_to_dfs(films_data)

In [277]:
df_gdata = (
    all_dfs_dict['df_gdata'][[
        'letterboxd_id',
        'letterboxd_shorttitle',
        'letterboxd_longtitle',
        'letterboxd_slug',
        'tmdb_id',
        'release_year',
        'duration',
        'avg_rating',
        'letterboxd_url',
        'tmdb_url',
        'imdb_url'
        ]]
    .astype({
        'release_year': 'int64',
        'duration': 'int64',
        'avg_rating': 'float64',
        'letterboxd_url': 'string',
        'tmdb_url': 'string',
        'imdb_url': 'string'
        })
    .reset_index(drop=True)
)

df_cast = (
    all_dfs_dict['df_cast'][[
        'film_id',
        'film_title',
        'name',
        'link',
        'character_name'
    ]]
    .assign(link = 'letterboxd.com' + all_dfs_dict['df_cast']['link'])
    .reset_index(drop=True)
    .astype({'link': 'string'})
)

df_crew = (
    all_dfs_dict['df_crew'][[
        'film_id',
        'film_title',
        'name',
        'role',
        'link',
    ]]
    .assign(link = 'letterboxd.com' + all_dfs_dict['df_crew']['link'])
    .reset_index(drop=True)
    .astype({'link': 'string'})
)

df_details = (
    all_dfs_dict['df_details'][[
        'film_id',
        'film_title',
        'key',
        'value',
        'link',
    ]]
    .assign(link = 'letterboxd.com' + all_dfs_dict['df_details']['link'])
    .reset_index(drop=True)
    .astype({'link': 'string'})
)

df_gthemes = (
    all_dfs_dict['df_gthemes'].rename(columns={0: 'value'})[[
        'film_id',
        'film_title',
        'value'
    ]]
    .reset_index(drop=True)
)

# Generate Analytical Dataframes

In [280]:
analytical_dataframes = {}

## General data

In [281]:
df_gdata['release_decade'] = (df_gdata['release_year'] // 10) * 10

In [189]:
#df_gdata.head(3)

In [282]:
median_rating = df_gdata['avg_rating'].median()
closest_to_median_row = df_gdata.iloc[(df_gdata['avg_rating'] - median_rating).abs().idxmin()]

main_metrics_dict = {
    'movies_watched': len(df_gdata),
    'minutes_watched': int(df_gdata['duration'].sum()),
    'hours_watched': float((df_gdata['duration'].sum() / 60).round(2)),
    'avg_movie_length': float(df_gdata['duration'].mean().round(2)),
    'name_longest_movie': df_gdata.loc[df_gdata['duration'].idxmax()]['letterboxd_shorttitle'],
    'duration_longest_movie': int(df_gdata['duration'].max().round(2)),
    'name_shortest_movie': df_gdata.loc[df_gdata['duration'].idxmin()]['letterboxd_shorttitle'],
    'duration_shortest_movie': int(df_gdata['duration'].min().round(2)),
    'avg_lbxd_rating': float(df_gdata['avg_rating'].mean().round(2)),
    'best_lbxd_rating': df_gdata.loc[df_gdata['avg_rating'].idxmax()]['letterboxd_shorttitle'],
    'worst_lbxd_rating': df_gdata.loc[df_gdata['avg_rating'].idxmin()]['letterboxd_shorttitle']
}

analytical_dataframes['main_metrics'] = pd.DataFrame([main_metrics_dict])

#analytical_dataframes['main_metrics']

In [283]:
adf_moviesperdecade = df_gdata.groupby('release_decade')[['letterboxd_id']].count().reset_index().rename(columns={'letterboxd_id': 'movie_count'})
new_row = pd.DataFrame({'release_decade': [1960], 'movie_count': [0]})

analytical_dataframes['movies_per_release_decade'] = (
    pd.concat([adf_moviesperdecade, new_row])
    .sort_values('release_decade')
    .reset_index(drop=True)
)

#analytical_dataframes['movies_per_release_decade']

## Cast

In [284]:
analytical_dataframes['popular_actors'] = (
    df_cast
    .groupby(['link', 'name'])[['film_id']].count().reset_index()
    .rename(columns={'film_id': 'movie_count'})
    .sort_values('movie_count', ascending=False)
    .query(" movie_count > 2 ")
    .reset_index(drop=True)
)[['name', 'movie_count', 'link']]

#analytical_dataframes['popular_actors']

In [285]:
most_popular_actors = list(analytical_dataframes['popular_actors']['name'])

analytical_dataframes['popular_actors_movies'] = (
    df_cast[['name', 'film_title']]
    .loc[df_cast['name'].isin(most_popular_actors)]
    .sort_values(['name', 'film_title'], ascending=[False, True])
)

#analytical_dataframes['popular_actors_movies']


## Crew

In [286]:
adf_crew_moviesperrole = (
    df_crew
    .groupby(['link', 'name', 'role'])[['film_id']].count().reset_index()
    .rename(columns={'film_id': 'movie_count'})
    .sort_values(['role', 'movie_count'], ascending=[True,False])
    .query(" movie_count > 1 ")
    .reset_index(drop=True)
)[['name', 'role', 'movie_count', 'link']]

#adf_crew_moviesperrole

In [287]:
main_roles = ['director', 'producer', 'writer']
secondary_roles = ['executive-producer', 'editor', 'cinematography', 'composer']

for role in (main_roles + secondary_roles):
    analytical_dataframes[f'popular_{role}s'] = adf_crew_moviesperrole.query(f" role == '{role}' ")

    most_popular_in_role = list(analytical_dataframes[f'popular_{role}s']['name'])

    analytical_dataframes[f'popular_{role}s_movies'] = (
        df_crew[['name', 'role', 'film_title']]
        .loc[df_crew['name'].isin(most_popular_in_role)]
        .query(f" role == '{role}' ")
        .sort_values(['name', 'film_title'], ascending=[False, True])
        .drop(columns='role')
    )

In [235]:
#analytical_dataframes['popular_executive-producers']
#analytical_dataframes['popular_executive-producers_movies']

In [288]:
adf_terciaryroles = (
    adf_crew_moviesperrole
    .loc[~adf_crew_moviesperrole['role'].isin(main_roles+secondary_roles)]
    .sort_values(['movie_count', 'role'], ascending=[False, True])
)

#adf_terciaryroles

In [313]:
terciary_crew = [170, 121, 270, 13, 77, 6, 36, 115]
most_popular_crew = adf_terciaryroles.loc[terciary_crew]['link']

analytical_dataframes['popular_crew_movies'] = (
    df_crew[['name', 'role', 'film_title']]
    .loc[df_crew['link'].isin((most_popular_crew))]
    .sort_values(['name', 'film_title'], ascending=[False, True])
)

#analytical_dataframes['popular_crew_movies']

## Details

In [222]:
df_details = df_details.drop_duplicates(subset=['film_id', 'film_title', 'key', 'value', 'link'], keep='first').reset_index(drop=True)

In [225]:
df_details['movie_count'] = df_details.groupby('link')['link'].transform('count')

In [232]:
df_studios = (
    df_details
    .loc[df_details['key'] == 'studio']
    .sort_values(['movie_count', 'value'], ascending=False)
    .rename(columns={'value': 'studio'})
)

df_countries = (
    df_details
    .loc[df_details['key'] == 'country']
    .sort_values(['movie_count', 'value'], ascending=False)
    .rename(columns={'value': 'country'})
)

df_languages = (
    df_details
    .loc[df_details['key'] == 'language']
    .sort_values(['movie_count', 'value'], ascending=False)
    .rename(columns={'value': 'language'})
)

In [237]:
df_studios.loc[df_studios['movie_count'] > 3]

,film_id,film_title,key,studio,link,movie_count
0,51869,The Wizard,studio,Universal Pictures,/studio/universal-pictures/,7
68,51986,Twelve Monkeys,studio,Universal Pictures,/studio/universal-pictures/,7
88,465978,Cats,studio,Universal Pictures,/studio/universal-pictures/,7
128,49632,Flash Gordon,studio,Universal Pictures,/studio/universal-pictures/,7
176,51386,Army of Darkness,studio,Universal Pictures,/studio/universal-pictures/,7
272,51338,An American Werewolf in London,studio,Universal Pictures,/studio/universal-pictures/,7
307,575258,Renfield,studio,Universal Pictures,/studio/universal-pictures/,7
27,48018,Speed Racer,studio,Warner Bros. Pictures,/studio/warner-bros-pictures/,5
43,46759,Cobra,studio,Warner Bros. Pictures,/studio/warner-bros-pictures/,5
64,2685,Unforgiven,studio,Warner Bros. Pictures,/studio/warner-bros-pictures/,5


In [245]:
df_countries.groupby('country')[['movie_count']].max().reset_index().sort_values('movie_count', ascending=False)

,country,movie_count
13,USA,59
12,UK,11
4,Hong Kong,5
0,Canada,4
1,China,3
2,France,3
3,Germany,3
8,Japan,2
11,South Korea,2
5,India,1


In [246]:
df_languages.groupby('language')[['movie_count']].max().reset_index().sort_values('movie_count', ascending=False)

,language,movie_count
3,English,65
4,French,7
2,Chinese,6
11,Japanese,6
5,German,4
16,Spanish,4
1,Cantonese,3
10,Italian,3
0,Arabic,2
12,Korean,2


## Genres and Themes

In [249]:
df_gthemes['movie_count'] = df_gthemes.groupby('value')['value'].transform('count')

In [269]:
genres = [
    'Adventure',
    'Family',
    'Drama',
    'Comedy',
    'Fantasy',
    'Action',
    'Horror',
    'Mystery',
    'Thriller',
    'Science Fiction',
    'Crime',
    'Western',
    'Animation',
    'History',
    'Romance',
    'Music'
]

df_genres = df_gthemes.loc[df_gthemes['value'].isin(genres)].rename(columns={'value': 'genre'})
df_themes = df_gthemes.loc[~df_gthemes['value'].isin(genres)].rename(columns={'value': 'theme'})

In [275]:
df_genres.groupby('genre')[['movie_count']].max().reset_index().sort_values('movie_count', ascending=False)

,genre,movie_count
0,Action,28
3,Comedy,26
9,Horror,22
14,Thriller,20
13,Science Fiction,19
1,Adventure,16
5,Drama,14
4,Crime,10
7,Fantasy,9
11,Mystery,6


In [277]:
df_themes.groupby('theme')[['movie_count']].max().reset_index().sort_values('movie_count', ascending=False).head(5)

,theme,movie_count
40,"Horror, the undead and monster classics",21
23,Epic heroes,20
15,Crude humor and satire,20
65,Superheroes in action-packed battles with vill...,15
46,Intense violence and sexual transgression,14


# Tests